In [ ]:
import tweepy
from tweepy import OAuthHandler
 
ACCESS_TOKEN = "###..."
ACCESS_TOKEN_SECRET = "###..."
CONSUMER_KEY = "###..."

CONSUMER_SECRET = "###..."
 
auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
 
api = tweepy.API(auth)

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(15):
    # Process a single status
    print(status.text)

In [ ]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    process_or_store(status._json)

In [ ]:
import json
def process_or_store(tweet):
        print(json.dumps(tweet))

In [ ]:
for friend in tweepy.Cursor(api.friends).items():
    process_or_store(friend._json)

In [ ]:
for tweet in tweepy.Cursor(api.user_timeline).items(10):
    process_or_store(tweet.text)

In [ ]:
##In case we want to “keep the connection open”, and gather all the upcoming tweets about a particular event,
##the streaming API is what we need from tweepy import Stream

from tweepy.streaming import StreamListener
class MyListener(StreamListener):
 
    def on_data(self, data):
        try :
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#CAF'])
import time
from datetime import timedelta
start_time = time.monotonic()
end_time = start_time + 23
print(timedelta(seconds=end_time - start_time))

In [ ]:
import time
print(type(time.monotonic()))

In [ ]:
import json
 
with open('mytweets.json', 'r') as f:
    line = f.readline() # read only the first tweet/line
    tweet = json.loads(line) # load it as Python dict
    print(json.dumps(tweet, indent=4)) 

In [ ]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 

In [ ]:
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

In [ ]:
import json
with open('mytweets.json', 'r') as f:
    for line in f:
        if line.strip():
               tweet = json.loads(line)
        tokens = preprocess(tweet['text'])
        print(json.dumps(tokens, indent=4))
        

In [ ]:
##Term frequencies
import operator 
import json
from collections import Counter
 
fname = 'mytweets.json'
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        if line.strip():
               tweet = json.loads(line)
        # Create a list with all the terms
        terms_all = [term for term in preprocess(tweet['text'])]
        # Update the counter
        count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [ ]:
print(stop)

In [ ]:
terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
print(terms_stop)

In [ ]:
# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)
# Count hashtags only
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs

In [ ]:
terms_hash

In [ ]:
## list of tuples using adjacent tokens
from nltk import bigrams 
 
terms_bigram = bigrams(terms_stop)

In [ ]:
!pip install vincent

In [ ]:
import vincent
 
word_freq = terms_only.most_common(20)
labels, freq = zip(*word_freq)
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.to_json('term_freq.json')